# 03 · Tokenization & Packing

Tokenize canonical BHC records using Unsloth's 4-bit Llama tokenizer and pack sequences.

In [1]:
# Persistent Drive + run mode setup
import os
import sys
from pathlib import Path

try:
    from google.colab import drive  # type: ignore
    DRIVE_MOUNT = Path('/content/drive')
    if not DRIVE_MOUNT.exists():
        drive.mount('/content/drive')
except Exception as exc:  # pragma: no cover
    print(f'Colab drive mount skipped: {exc}')

if Path('/content/drive').exists():
    DRIVE_ROOT = Path('/content/drive/MyDrive').resolve()
else:
    DRIVE_ROOT = Path.home().resolve()

PROJECT_ROOT = DRIVE_ROOT / 'secure-llm-mia'
if not PROJECT_ROOT.exists():
    raise FileNotFoundError('Run 00_colab_setup.ipynb first to clone the repo on Drive.')

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

os.environ['SECURE_LLM_MIA_ROOT'] = str(PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

from src.utils.runtime import current_run_mode

RUN_MODE = current_run_mode()
print('PROJECT_ROOT:', PROJECT_ROOT)
print('Active run mode:', RUN_MODE.name, '-', RUN_MODE.description)

DATA_ROOT = PROJECT_ROOT / 'data'
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'
CHECKPOINT_ROOT = PROJECT_ROOT / 'checkpoints'
for path in (DATA_ROOT, ARTIFACTS_DIR, CHECKPOINT_ROOT):
    path.mkdir(parents=True, exist_ok=True)

BHC_DATA_DIR = DRIVE_ROOT / 'mimic-iv-bhc'
BHC_DATA_DIR.mkdir(parents=True, exist_ok=True)
BHC_CSV_PATH = BHC_DATA_DIR / 'mimic-iv-bhc.csv'
print('BHC CSV path:', BHC_CSV_PATH)


Mounted at /content/drive
PROJECT_ROOT: /content/drive/MyDrive/secure-llm-mia
Active run mode: subset - Quick debugging subset (<=2k rows) for lightweight Colab smoke tests.
BHC CSV path: /content/drive/MyDrive/mimic-iv-bhc/mimic-iv-bhc.csv


In [2]:
# Install unsloth if not already installed
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/

In [3]:
import pandas as pd
from datasets import Dataset

from unsloth import FastLanguageModel

SLICED_DATASET_PATH = ARTIFACTS_DIR / f'sliced_dataset_{RUN_MODE.name}.parquet'
TARGET_SLICES = 4
TRAIN_LIMIT = 1000
MAX_SEQ_LENGTH = 4096

if not SLICED_DATASET_PATH.exists():
    raise FileNotFoundError('Run notebook 02 to create the sliced dataset first.')

sliced_df = pd.read_parquet(SLICED_DATASET_PATH)
if 'slice_id' not in sliced_df.columns:
    raise KeyError('sliced_dataset is missing `slice_id`. Re-run notebook 02 to rebuild it.')

slice_ids = sorted(int(s) for s in sliced_df['slice_id'].dropna().unique())[:TARGET_SLICES]
print('Available slice IDs:', slice_ids)


==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

Tokenizer eos_token: <|end_of_text|>


In [4]:
MODEL_NAME = os.getenv('UNSLOTH_MODEL_NAME', 'unsloth/Meta-Llama-3.1-8B-bnb-4bit')
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)
tokenizer.model_max_length = MAX_SEQ_LENGTH
del model
print('Tokenizer eos_token:', tokenizer.eos_token)


Created 1738 packed sequences for run mode subset.
Max packed length: 9477


Export per-slice tokenized datasets for fine-tuning.

In [5]:
from pathlib import Path

PACKED_DIR = ARTIFACTS_DIR / 'packed' / RUN_MODE.name
PACKED_DIR.mkdir(parents=True, exist_ok=True)
summary_rows = []

for slice_id in slice_ids:
    slice_df = sliced_df[(sliced_df['slice_id'] == slice_id) & (sliced_df['split_tag'] == 'train')]
    if slice_df.empty:
        print(f'Slice {slice_id}: no training rows found, skipping.')
        continue
    sampled = slice_df.sample(frac=1.0, random_state=17).head(TRAIN_LIMIT)
    tokenized = tokenizer(
        sampled['text'].tolist(),
        return_attention_mask=True,
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        padding='max_length',
    )
    dataset = Dataset.from_dict(
        {
            'input_ids': tokenized['input_ids'],
            'attention_mask': tokenized['attention_mask'],
            'example_id': sampled['subject_id'].astype(str).tolist(),
        }
    )
    out_path = PACKED_DIR / f'slice_{slice_id}.parquet'
    dataset.to_parquet(str(out_path))
    summary_rows.append({'slice_id': int(slice_id), 'examples': len(dataset)})
    print(f'Slice {slice_id}: saved {len(dataset)} sequences to {out_path}')

if summary_rows:
    summary_df = pd.DataFrame(summary_rows)
    print(summary_df)
else:
    print('No slices were processed. Check the sliced dataset and RUN_MODE settings.')


Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Saved packed dataset to /content/drive/MyDrive/secure-llm-mia/artifacts/packed/subset/packed_sequences.parquet
Saved summary metadata to /content/drive/MyDrive/secure-llm-mia/artifacts/packed/subset/packed_summary.parquet
